In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from time import sleep
import pandas as pd
import time
import re
from datetime import date, timedelta


REMOVE_ADS_SCRIPT = """
var ads = document.querySelectorAll('[id^="google_ads"], [class*="ads"], [class*="sponsored"]');
ads.forEach(ad => ad.remove());
"""


def start_google_chrome(url, screen_width=2200, screen_height=1900):
    """
    Undetected Chrome を起動し、指定URLにアクセス
    - ウィンドウを左半分に配置
    - 広告対策オプションを追加
    :param url: 開くURL
    :param screen_width: 画面の横幅 (デフォルト: 2200)
    :param screen_height: 画面の高さ (デフォルト: 1900)
    :return: WebDriver インスタンス
    """

    # Chromeオプションを設定
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")  # セキュリティサンドボックス無効化
    options.add_argument(
        "--disable-blink-features=AutomationControlled"
    )  # ボット検出回避
    options.add_argument("--disable-extensions")  # 拡張機能無効化
    options.add_argument("--disable-popup-blocking")  # ポップアップブロック無効化
    options.add_argument("--disable-gpu")  # GPUレンダリング無効化
    options.add_argument("--start-maximized")  # 最大化
    options.add_argument(
        "--disable-infobars"
    )  # "Chromeは自動テストソフトウェアによって制御されています"を非表示
    options.add_argument("--disable-notifications")  # 通知をブロック
    options.add_argument("--remote-debugging-port=9222")  # デバッグ用ポート

    # Chromeドライバーの起動
    driver = uc.Chrome(options=options)

    # 画面の左半分にウィンドウを配置
    driver.set_window_position(0, 0)  # 左上 (0,0) に配置
    driver.set_window_size(screen_width // 2, screen_height)  # 画面の半分の幅に設定

    # 指定URLを開く
    driver.get(url)
    print("-" * 50)
    print(f"実行成功: {url}")
    driver.implicitly_wait(5)  # 最大30秒待機
    time.sleep(1)  # 読み込み待機

    return driver  # WebDriver インスタンスを返す


def click_date_link(driver, DAYS_AGO, MAX_RETRIES=3):
    """
    指定したDAYS_AGOの日付リンクをクリックし、ページ遷移を確認する
    :param driver: Selenium WebDriver
    :param DAYS_AGO: 過去の日数（1=最新の日付）
    :param MAX_RETRIES: 最大リトライ回数
    :return: 成功した場合はTrue、失敗した場合はFalse
    """
    retries = 0
    current_url = driver.current_url

    try:
        wait = WebDriverWait(driver, 10)
        date_links = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".table-data-cell a"))
        )

        if DAYS_AGO > len(date_links):
            print(f"エラー: DAYS_AGO={DAYS_AGO} に対応するリンクがありません。")
            return False

        target_link = date_links[DAYS_AGO - 1]
        print(f"クリック対象リンク: {target_link.text}")

        while retries < MAX_RETRIES:
            driver.execute_script(
                "arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});",
                target_link,
            )
            time.sleep(5)  # スクロール後の待機
            wait.until(EC.element_to_be_clickable(target_link))
            time.sleep(5)

            target_link.click()

            wait.until(EC.url_changes(current_url))
            time.sleep(5)

            if "google_vignette" in driver.current_url:
                retries += 1
                print(
                    f"⚠ Google広告が表示されました。リトライします ({retries}/{MAX_RETRIES})"
                )
                driver.back()  # 広告ページから戻る
                time.sleep(5)
                continue  # リトライ

            print(f"遷移成功: {driver.current_url}")
            date, hall_name = extract_date_hall(driver.current_url)
            return date, hall_name

    except Exception as e:
        print(f"エラー発生: {e}")

    print("クリック失敗: リトライ回数超過")
    return False  # 失敗


def extract_date_hall(url):
    """
    URL から日付 (YYYY-MM-DD) とホール名を抽出
    :param url: 解析するURL
    :return: (date, hall_name) のタプル
    """
    pattern = r"https://ana-slo\.com/(\d{4})-(\d{1,2})-(\d{1,2})-(.+)-data/"
    match = re.search(pattern, url)

    if match:
        year, month, day, hall_name = match.groups()
        date = f"{year}-{int(month):02d}-{int(day):02d}"  # 月日をゼロ埋め
        hall_name = hall_name.replace(
            "-", " "
        ).upper()  # "-" をスペースに置き換えて大文字に
        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")
        return date, hall_name
    else:
        return None, None  # マッチしない場合


def click_machine_by_name(driver):
    """
    指定した機種名を検索し、該当する要素をクリック
    :param driver: Selenium WebDriver
    :param search_machine: 検索する機種名
    :return: クリック成功: True / 失敗: False
    """
    try:
        title = driver.title
        date_pattern = r"\d{4}/\d{2}/\d{2}"
        date_match = re.search(date_pattern, title)
        date = date_match.group().replace("/", "-") if date_match else None
        hall_pattern = r"\d{4}/\d{2}/\d{2} (.+?) データまとめ"
        hall_match = re.search(hall_pattern, title)
        hall_name = hall_match.group(1).replace(" ", "_") if hall_match else None

        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")

        return date, hall_name

    except Exception as e:
        print(f"❌ エラー発生: {e}")
        return False


def extract_and_save_model_data(driver, hall_name, date):
    """
    指定した機種のデータを抽出し、CSVに保存する
    :param driver: Selenium WebDriver
    :param section_num: セクション番号 (該当機種のインデックス)
    :param hall_name: ホール名
    :param date: 日付 (yyyy-mm-dd)
    :return: 成功時 True, 失敗時 False
    """
    try:
        # すべての機種を表示
        button = driver.find_element(By.ID, "all_data_btn")
        button.click()

        # データ行を取得
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, f"#all_data_table_wrapper > div.dataTables_scroll tr")
            ))

        if not rows:
            print(f"⚠️ データが見つかりません")
            # return False

        print(f"{len(rows)} row 取得開始")
        print("取得中...")

        # ヘッダー取得
        header_cells = rows[0].find_elements(By.TAG_NAME, "th")
        columns = [cell.text for cell in header_cells]

        # データ取得
        data = []
        for row in rows[2:]:  # 最終行を除外
            cells = row.find_elements(By.TAG_NAME, "td")
            data.append([cell.text for cell in cells])
        print(f": {len(data)} data 取得完了")

        # DataFrameを作成しCSVに保存
        df = pd.DataFrame(data, columns=columns)
        file_name = f"../csv/{hall_name}_{date}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")

        print(rows[1].text)

        print(f"データ保存完了: {file_name}")
        return True

    except Exception as e:
        print(f"エラー発生: {e}")
        return False

In [5]:
# driver = start_google_chrome("https://google.com")

# for days_ago in range(DAYS_AGO, DAYS_AGO+RERIOD):
        
#     driver.get(url)
#     driver.execute_script(REMOVE_ADS_SCRIPT)
    
#     click_date_link(driver, days_ago, MAX_RETRIES=MAX_RETRIES)
#     driver.execute_script(REMOVE_ADS_SCRIPT)

#     date, hall_name = click_machine_by_name(driver)
#     extract_and_save_model_data(driver, hall_name, date)
    
#     sleep(5)

# driver.close()

## テストコード 手動用

In [6]:
DATA = "ホールデータ"
EXTRA_WORD = "-データ一覧"
PREF = "東京都"
PREF = "埼玉県"
HALL_NAME = "ニュークラウン川越2号店"
url = f"https://ana-slo.com/{DATA}/{PREF}/{HALL_NAME}{EXTRA_WORD}/"

In [7]:
driver = start_google_chrome("https://google.com")
driver.get(url)
driver.execute_script(REMOVE_ADS_SCRIPT)

--------------------------------------------------
実行成功: https://google.com


In [ ]:
DAYS_AGO = 158
date, hall_name = click_date_link(driver, DAYS_AGO, MAX_RETRIES=5)

In [18]:
# すべての機種表示
driver.execute_script(REMOVE_ADS_SCRIPT)
button = driver.find_element(By.ID, "all_data_btn")
button.click()

In [19]:
rows = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, f"#all_data_table_wrapper > div.dataTables_scroll tr")
    ))
if not rows:
    print(f"⚠️ データが見つかりません")

header_cells = rows[0].find_elements(By.TAG_NAME, "th")
columns = [cell.text for cell in header_cells]
data = []
for row in rows[2:]:  # 最終行を除外
    cells = row.find_elements(By.TAG_NAME, "td")
    data.append([cell.text for cell in cells])
df = pd.DataFrame(data, columns=columns)

# DataFrameを作成しCSVに保存
df = pd.DataFrame(data, columns=columns)
rename_columns ={"機種名": "model_name", "台番号": "unit_no", "G数": "game", "差枚": "medals"}
df.rename(columns=rename_columns, inplace=True)
cols_to_convert = ["unit_no", "game", "medals", "BB", "RB"]
for col in cols_to_convert:
    if col in df.columns:
        df[col] = df[col].replace(",", "", regex=True).astype(int)
    else:
        df[col] = 0
        
# 同じ機種を指しているが表記ゆれ対策
alias_map = {
    "SミスタージャグラーKK": "ミスタージャグラー",
    "S ミスタージャグラー KK": "ミスタージャグラー",
}
df["model_name"] = df["model_name"].replace(alias_map)

file_name = f"C:/python/dataOnline/anaslo_02/csv/{PREF}_{HALL_NAME}_{date}.csv"
df.to_csv(file_name, index=False, encoding="utf-8-sig")

df.head()

,model_name,unit_no,game,medals,BB,RB,合成確率,BB確率,RB確率
0,アイムジャグラーEX-TP,331,2671,-335,9,5,1/190.8,1/296.8,1/534.2
1,アイムジャグラーEX-TP,379,2300,-68,7,9,1/143.8,1/328.6,1/255.6
2,アイムジャグラーEX-TP,380,4943,1170,24,12,1/137.3,1/206.0,1/411.9
3,アイムジャグラーEX-TP,381,3129,404,15,5,1/156.4,1/208.6,1/625.8
4,アイムジャグラーEX-TP,382,6422,1620,25,30,1/116.8,1/256.9,1/214.1
